In [152]:
import random
from tqdm import tqdm
import pandas as pd
import numpy as np

## Blackjack Deck Simulation: `deck_50perc()` Function

The `deck_50perc()` function creates a simulated deck with 50% of its cards randomly removed. Here's a breakdown of the function:

- **Parameters**:
  - `deck_count`: Specifies the number of decks being used (in this case, 5).
  - `cards`: List of possible card values, representing cards from 2 through 10 and face cards ('K', 'Q', 'J', 'A').
  - `deck`: Constructs a full deck with each card appearing four times (standard suit distribution) and multiplied by `deck_count`.

- **Random Removal**:
  - Using `random.sample()`, the function randomly removes half of the cards from `deck`.
  - `deck.remove(i)`: Removes each selected card from `deck` until half of the deck is gone.


In [2]:
def deck_50perc():
    deck_count = 5
    cards = list(range(2,11))+['K','Q','J','A']
    deck = cards*4*deck_count
    for i in random.sample(deck,deck_count*52//2):
        deck.remove(i)
    return deck

In [3]:
deck = deck_50perc()

In [4]:
len(deck)

130

## Blackjack Card Points: `points` Dictionary

The `points` dictionary is used to assign values to each card in a blackjack deck. This dictionary comprehension assigns points based on standard blackjack rules:

- **Ace ('A')**: Has a dual value of either 1 or 11, represented by `(1, 11)`.
- **Face Cards ('K', 'Q', 'J') and 10**: All have a value of 10.
- **Number Cards (2 through 9)**: Each card has its face value (e.g., 2, 3, ..., 9).


In [5]:
points = {i: (1, 11) if i == 'A' else 10 if i in ['K', 'Q', 'J', 10] else i for i in list(range(2,11))+['K','Q','J','A']}

In [6]:
points

{2: 2,
 3: 3,
 4: 4,
 5: 5,
 6: 6,
 7: 7,
 8: 8,
 9: 9,
 10: 10,
 'K': 10,
 'Q': 10,
 'J': 10,
 'A': (1, 11)}

In [7]:
list(points.keys())

[2, 3, 4, 5, 6, 7, 8, 9, 10, 'K', 'Q', 'J', 'A']

## Blackjack Card Draw: `hit()` Function

The `hit()` function simulates drawing a card from the deck and retrieves its corresponding point value based on blackjack rules.

- **Process**:
  - `card = random.choice(deck)`: Randomly selects a card from the `deck`.
  - `return (card, points[card])`: Returns a tuple with the card and its point value from the `points` dictionary.

The function provides a quick way to simulate a hit action in blackjack, giving both the card drawn and its value for scoring purposes.


In [8]:
def hit():
    card = random.choice(deck)
    return (card, points[card])

In [9]:
hit()

(4, 4)

In [10]:
def get_a_pair():
    x,y=None,None
    while True:
        x,y = random.choice(list(points.keys())),random.choice(list(points.keys()))
        if x=='A' and y =='A':
            return (x,y,2,12)
        elif x=='A':
            return (x,y,points[y]+1,points[y]+11)
        elif y=='A':
            return (x,y,points[x]+1,points[x]+11)
        else:
            return (x,y,points[x]+points[y])
        break

In [11]:
def play_12():
    while True:
        res = get_a_pair()
        if res==('A','A',2,12):
            continue
        elif (len(res)==3 and res[2]==12): 
            break
        elif len(res)==4 and (res[2]==12 or res[3]==12):
            break
    return res

In [12]:
play_12()

(3, 9, 12)

In [13]:
def play_10x():
    while True:
        res = get_a_pair()
        if 21 in res:
            continue
        if res[0] in ['K', 'Q', 'J', 10]:
            break
    return res    

In [14]:
play_10x()

('J', 9, 19)

In [108]:
possible_dealer_hands=set()
for i in ['K', 'Q', 'J', 10]:
    for j in [2,3,4,5,6,7,8,9,10,'K', 'Q', 'J']:
        possible_dealer_hands.add((i,j,points[i]+points[j]))
        possible_dealer_hands.add((j,i,points[i]+points[j]))

In [98]:
possible_player_hands = set()
for i in range(2,11):
    possible_player_hands.add((i,12-i,12))
for i in ['K','Q','J']:
    possible_player_hands.add((i,2,12))
    possible_player_hands.add((2,i,12))

In [16]:
def remove_from_deck(items):
    for i in items:
        deck.remove(i)

In [17]:
def play_12_vs_10x(dealer):
    player = play_12()
    remove_from_deck([dealer[0],dealer[1],player[0],player[1]])
    return dealer,player

In [18]:
dealer,player = play_12_vs_10x(random.choice(possible_dealer_hands))
print('dealer',dealer)
print('player',player)

dealer ('K', 10, 20)
player (8, 4, 12)


In [19]:
len(deck)

126

In [20]:
def player_bust(tot_pts):
    return tot_pts>21

In [21]:
def decide_winner(player_pts,dealer_pts):
    if player_pts > dealer_pts:
        return 1
    elif player_pts == dealer_pts:
        return 0
    else:
        return -1

In [26]:
exp=1000

In [120]:
data=[] #15*
for player in tqdm(possible_player_hands):
    for dealer in possible_dealer_hands:
        dealer_win_freq = 0
        player_win_freq = 0
        draw_freq = 0
        # log = ''
        for i in range(exp):
            deck = deck_50perc()
            did_player_bust = -1
            tot_pts = player[-1]
            while tot_pts<17:
                player_card = hit()
                # log+=str(player_card) + ' '
                remove_from_deck([player_card[0]])
                if player_card[0] == 'A':
                    tot_pts+=1
                else:
                    tot_pts+=player_card[1]
                    if player_bust(tot_pts):
                        did_player_bust=1
            if did_player_bust==1:
                # log+="dwb pb"
                dealer_win_freq+=1
            elif decide_winner(tot_pts, dealer[-1])==-1:
                # log+="dwb pLH"
                dealer_win_freq+=1
            elif decide_winner(tot_pts,dealer[-1])==0:
                # log+="draw"
                draw_freq+=1
            else:
                # log+="player HH"
                player_win_freq+=1 
        data.append({'dealer':dealer, 'player':player, 'dealer_win':dealer_win_freq,'player_win':player_win_freq, 'push':draw_freq})
            # log+=" | "

100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [02:59<00:00, 11.96s/it]


In [122]:
df = pd.DataFrame(data)

In [126]:
df[['dealer_win', 'player_win', 'push']] = df[['dealer_win', 'player_win', 'push']] / exp * 100

In [127]:
df.dealer_win.mean()

np.float64(57.62399999999999)

In [128]:
df.player_win.mean()

np.float64(37.21375)

In [129]:
df.push.mean()

np.float64(5.16225)

In [130]:
df

,dealer,player,dealer_win,player_win,push
0,"(J, 8, 18)","(K, 2, 12)",58.7,31.6,9.7
1,"(10, 3, 13)","(K, 2, 12)",47.9,52.1,0.0
2,"(J, 4, 14)","(K, 2, 12)",45.2,54.8,0.0
3,"(Q, K, 20)","(K, 2, 12)",77.5,11.0,11.5
4,"(5, Q, 15)","(K, 2, 12)",48.3,51.7,0.0
...,...,...,...,...,...
1195,"(Q, 2, 12)","(4, 8, 12)",46.9,53.1,0.0
1196,"(7, K, 17)","(4, 8, 12)",46.8,40.9,12.3
1197,"(2, J, 12)","(4, 8, 12)",50.5,49.5,0.0
1198,"(4, J, 14)","(4, 8, 12)",48.3,51.7,0.0


In [131]:
df.to_csv('stats.csv', index=False)

In [132]:
len(possible_dealer_hands)*len(possible_player_hands)*exp

1200000

In [148]:
len(df)


1200